In [1]:
import pandas as pd
from google.cloud import bigquery
import pickle
import os

In [96]:
client = bigquery.Client()
merch_data_ref = client.dataset('merch_store', project = 'dummy24571')

In [97]:
def bq2df(sql):
    query = client.query(sql) # API request
    results = query.result()
    return results.to_dataframe()

In [98]:
q = ('SELECT * FROM `dummy24571.merch_store.ga_sessions_*` '
    'LIMIT 100')

In [99]:
complete_data = bq2df(q)

In [100]:
complete_data.head()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,NaN,1,1478887689,1478887689,20161111,"{'visits': 1, 'hits': 5, 'pageviews': 4, 'time...",{'referralPath': '/Where-can-I-buy-a-stuffed-G...,"{'browser': 'Edge', 'browserVersion': 'not ava...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 10, 'minu...",5182045908444892546,None,Social,Not Socially Engaged
1,NaN,2,1478872764,1478872764,20161111,"{'visits': 1, 'hits': 6, 'pageviews': 6, 'time...",{'referralPath': '/Where-can-I-buy-a-stuffed-G...,"{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",8659416185418764904,None,Social,Not Socially Engaged
2,NaN,1,1478871017,1478871017,20161111,"{'visits': 1, 'hits': 3, 'pageviews': 3, 'time...",{'referralPath': '/r/golang/comments/4zk0qy/in...,"{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'North...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",1960184141603407399,None,Social,Not Socially Engaged
3,NaN,1,1478872521,1478872521,20161111,"{'visits': 1, 'hits': 4, 'pageviews': 4, 'time...",{'referralPath': '/r/golang/comments/4spnk9/wh...,"{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",8659416185418764904,None,Social,Not Socially Engaged
4,NaN,1,1478890578,1478890578,20161111,"{'visits': 1, 'hits': 11, 'pageviews': 11, 'ti...",{'referralPath': '/r/golang/comments/4spnk9/wh...,"{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 10, 'minu...",1292675398602544083,None,Social,Not Socially Engaged


In [78]:
merch_data = client.get_dataset(merch_data_ref)

In [79]:
merch_data

Dataset(DatasetReference('dummy24571', 'merch_store'))

In [80]:
tables = [x.table_id for x in client.list_tables(merch_data)]

In [81]:
len(tables)

367

In [82]:
merch_full = client.get_table(merch_data.table('ga_sessions_20160801'))

In [83]:
merch_full

Table(TableReference(DatasetReference('dummy24571', 'merch_store'), 'ga_sessions_20160801'))

In [28]:
# [command for command in dir(merch_full) if not command.startswith('_')]

In [12]:
# merch_full.schema

In [13]:
query = """
  SELECT
      fullVisitorId AS visitor_id,
      visitId AS visit_id,
      CONCAT(fullVisitorId, CAST(visitId AS STRING)) AS unique_session_id,
      date,
      product.v2ProductCategory AS product_category,
      product.v2ProductName AS product_name,
      product.productSKU AS product_sku,
      product.productPrice/1e6 AS product_price,
      product.productQuantity AS product_quantity,
      product.productRevenue/1e6 AS product_revenue,
      totals.totalTransactionRevenue/1e6 AS total_revenue
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
      , UNNEST(hits) AS hits
      , UNNEST(hits.product) AS product
  WHERE
      _TABLE_SUFFIX BETWEEN '20170401' AND '20170430'
      AND geoNetwork.country = 'United States'
      AND productRevenue IS NOT NULL
  ORDER BY
      4 ASC, 10 DESC
"""


In [14]:
data_0 = bq2df(query)

In [15]:
data_0

,visitor_id,visit_id,unique_session_id,date,product_category,product_name,product_sku,product_price,product_quantity,product_revenue,total_revenue
0,6818117303364696962,1491091099,68181173033646969621491091099,20170401,Apparel,Google Women's Fleece Hoodie,GGOEGAQB036014,44.79,1,47.040000,164.17
1,6818117303364696962,1491091099,68181173033646969621491091099,20170401,Apparel,Google Men's Zip Hoodie,GGOEGAFB035814,44.79,1,47.040000,164.17
2,6818117303364696962,1491091099,68181173033646969621491091099,20170401,Apparel,Google Youth Girl Hoodie,GGOEGAYQ068956,35.20,1,37.450000,164.17
3,6818117303364696962,1491091099,68181173033646969621491091099,20170401,Apparel,Google Toddler Hoodie Royal Blue,GGOEGAXC065629,30.39,1,32.640000,164.17
4,2418102200342405484,1491072529,24181022003424054841491072529,20170401,Apparel,Google Onesie Red,GGOEGAWR061148,23.99,1,25.990000,25.99
...,...,...,...,...,...,...,...,...,...,...,...
2772,1206882682891516620,1493605954,12068826828915166201493605954,20170430,Office,Recycled Paper Journal Set,GGOEGOCC017599,3.99,1,4.704285,48.38
2773,4859141843536295311,1493617449,48591418435362953111493617449,20170430,Waze,Waze Mood Original Window Decal,GGOEWFKA083199,1.59,1,2.732857,69.45
2774,4859141843536295311,1493617449,48591418435362953111493617449,20170430,Waze,Waze Mood Ninja Window Decal,GGOEWFKA083299,1.59,1,2.732857,69.45
2775,1206882682891516620,1493605954,12068826828915166201493605954,20170430,Office,Ballpoint Pen Blue,GGOEGOAC021799,0.99,2,2.694285,48.38


In [46]:
query = """
  SELECT
      fullVisitorId AS visitor_id,
      visitId AS visit_id,
      CONCAT(fullVisitorId, CAST(visitId AS STRING)) AS unique_session_id,
      date,
      product.v2ProductCategory AS product_category,
      product.v2ProductName AS product_name,
      product.productSKU AS product_sku,
      product.productPrice/1e6 AS product_price,
      totals.timeOnScreen AS time_spent,
      totals.transactions AS transection,
      product.productQuantity AS product_quantity,
      product.productRevenue/1e6 AS product_revenue,
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
      , UNNEST(hits) AS hits
      , UNNEST(hits.product) AS product
  WHERE
      _TABLE_SUFFIX BETWEEN '20170401' AND '20170531'
      AND geoNetwork.country = 'United States'
      AND productRevenue IS NOT NULL
  ORDER BY
      4 ASC
"""


In [47]:
data_1 = bq2df(query)

In [48]:
data_1

,visitor_id,visit_id,unique_session_id,date,product_category,product_name,product_sku,product_price,time_spent,transection,product_quantity,product_revenue
0,2556948849524550835,1491097844,25569488495245508351491097844,20170401,Apparel,Google Men's Short Sleeve Hero Tee Heather,GGOEGAAJ073415,18.99,NaN,1,1,23.99
1,1618884882553763938,1491090577,16188848825537639381491090577,20170401,Apparel,Google Men's 100% Cotton Short Sleeve Hero Tee...,GGOEGAAB010517,16.99,NaN,1,1,24.99
2,1639589047263376521,1491095245,16395890472633765211491095245,20170401,Apparel,Google Womens 3/4 Sleeve Baseball Raglan Heath...,GGOEGAEA030413,15.99,NaN,1,1,20.49
3,1639589047263376521,1491095245,16395890472633765211491095245,20170401,Apparel,Google Women's Short Sleeve Badge Tee Red Heather,GGOEGAER029213,15.19,NaN,1,1,19.69
4,2418102200342405484,1491072529,24181022003424054841491072529,20170401,Apparel,Google Onesie Red,GGOEGAWR061148,23.99,NaN,1,1,25.99
...,...,...,...,...,...,...,...,...,...,...,...,...
5911,8637158932301658318,1496250783,86371589323016583181496250783,20170531,Gift Cards,Gift Card - $25.00,GGOEGGCX056299,25.00,NaN,15,1,32.00
5912,8637158932301658318,1496250783,86371589323016583181496250783,20170531,Gift Cards,Gift Card - $25.00,GGOEGGCX056299,25.00,NaN,15,1,32.00
5913,8637158932301658318,1496250783,86371589323016583181496250783,20170531,Gift Cards,Gift Card - $25.00,GGOEGGCX056299,25.00,NaN,15,1,32.00
5914,8637158932301658318,1496250783,86371589323016583181496250783,20170531,Gift Cards,Gift Card - $25.00,GGOEGGCX056299,25.00,NaN,15,1,32.00


In [53]:
query = """
  SELECT
      CONCAT(fullVisitorId, CAST(visitId AS STRING)) AS unique_session_id,
      date,
      totals.totalTransactionRevenue/1e6 AS total_revenue
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
      _TABLE_SUFFIX BETWEEN '20170401' AND '20170430'
      AND geoNetwork.country = 'United States'
      AND totals.totalTransactionRevenue IS NOT NULL
"""

In [54]:
data_2 = bq2df(query)

In [55]:
data_2

,unique_session_id,date,total_revenue
0,23031991900246223211493140374,20170425,57.97
1,00322853888042345731493134510,20170425,146.50
2,95898221681013002091493127285,20170425,341.75
3,18544058375709823701493126454,20170425,54.67
4,60066570470165770711493151594,20170425,28.19
...,...,...,...
889,03799108209105408081491329068,20170404,34.49
890,04156086765464325591491340648,20170404,122.98
891,01029675510187575791491370615,20170404,61.93
892,46004361868375183581491361567,20170404,31.98


In [60]:
query = """
  SELECT
      fullVisitorId AS visitor_id,
      visitId AS visit_id,
      date,
      product.v2ProductCategory AS product_category,
      product.v2ProductName AS product_name,
      product.productSKU AS product_sku,
      product.productPrice/1e6 AS product_price,
      product.productQuantity AS product_quantity,
      product.productRevenue/1e6 AS product_revenue,
      totals.totalTransactionRevenue/1e6 AS total_revenue
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
      , UNNEST(hits) AS hits
      , UNNEST(hits.product) AS product
  WHERE
      _TABLE_SUFFIX BETWEEN '20170401' AND '20170430'
      AND geoNetwork.country = 'United States'
      AND productRevenue IS NOT NULL
  ORDER BY
      4 ASC, 10 DESC
"""


In [61]:
data_3 = bq2df(query)

In [79]:
data_3.shape

(2777, 10)

In [80]:
data_3.head()

,visitor_id,visit_id,date,product_category,product_name,product_sku,product_price,product_quantity,product_revenue,total_revenue
0,6104963094502152821,1493314922,20170427,${productitem.product.origCatName},Google Sunglasses,GGOEGHGC019799,2.80,13,36.733333,463.54
1,0608915197735218105,1492021292,20170412,${productitem.product.origCatName},Google Doodle Decal,GGOEGFKQ020799,2.99,4,12.498461,363.32
2,0717343852152056082,1492457532,20170417,${productitem.product.origCatName},Recycled Mouse Pad,GGOEGODR017799,1.20,6,7.800000,278.00
3,5263059809967138936,1491587620,20170407,${productitem.product.origCatName},Google Youth Baseball Raglan Heather/Black,GGOEGAYB068056,19.99,2,40.070909,259.52
4,8657427332734176422,1491585605,20170407,${productitem.product.origCatName},Recycled Mouse Pad,GGOEGODR017799,1.20,35,44.000000,241.30


In [16]:
q = """
    SELECT 
      CONCAT(fullVisitorID,'-',CAST(visitNumber AS STRING)) AS visitorId,
      hitNumber,
      time,
      page.pageTitle,
      type,
      productSKU,
      v2ProductName,
      v2ProductCategory,
      productPrice/1000000 as productPrice_USD

    FROM 
      `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`, 
      UNNEST(hits) AS hits,
      UNNEST(hits.product) AS hits_product

"""

In [17]:
dd = bq2df(q)

In [23]:
dd.head()

,visitorId,hitNumber,time,pageTitle,type,productSKU,v2ProductName,v2ProductCategory,productPrice_USD
0,7194065619159478122-1,2,30507,Gift Cards,PAGE,GGOEGGCX056299,Gift Card - $25.00,Home/Gift Cards/,25.00
1,7194065619159478122-1,2,30507,Gift Cards,PAGE,GGOEGGCX056499,Gift Card - $50.00,Home/Gift Cards/,50.00
2,7194065619159478122-1,2,30507,Gift Cards,PAGE,GGOEGGCX056199,Gift Card- $100.00,Home/Gift Cards/,100.00
3,7194065619159478122-1,2,30507,Gift Cards,PAGE,GGOEGGCX056399,Gift Card - $250.00,Home/Gift Cards/,250.00
4,8159312408158297118-151,2,289206,Men's-T-Shirts,PAGE,GGOEGAAX0104,Google Men's 100% Cotton Short Sleeve Hero Tee...,Home/Apparel/Men's/Men's-T-Shirts/,16.99


In [118]:
qq = """    
    CREATE OR REPLACE TABLE merch_store.aggregate_web_stats AS (
      WITH
        durations AS (
          --calculate pageview durations
          SELECT
            CONCAT(fullVisitorID,'-', 
                 CAST(visitNumber AS STRING),'-', 
                 CAST(hitNumber AS STRING) ) AS visitorId_session_hit,
            LEAD(time, 1) OVER (
              PARTITION BY CONCAT(fullVisitorID,'-',CAST(visitNumber AS STRING))
              ORDER BY
              time ASC ) - time AS pageview_duration
          FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_2016*`,
            UNNEST(hits) AS hit 
        ),

        prodview_durations AS (
          --filter for product detail pages only
          SELECT
            CONCAT(fullVisitorID,'-',CAST(visitNumber AS STRING)) AS visitorId,
            productSKU AS itemId,
            v2ProductName AS product_name,
            IFNULL(dur.pageview_duration,
              1) AS pageview_duration,
          FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_2016*` t,
            UNNEST(hits) AS hits,
            UNNEST(hits.product) AS hits_product
          JOIN
            durations dur
          ON
            CONCAT(fullVisitorID,'-',
                   CAST(visitNumber AS STRING),'-',
                   CAST(hitNumber AS STRING)) = dur.visitorId_session_hit
          WHERE
          #action_type: Product detail views = 2
          eCommerceAction.action_type = "2" 
        ),

        aggregate_web_stats AS(
          --sum pageview durations by visitorId, itemId
          SELECT
            visitorId,
            product_name,
            itemId,
            SUM(pageview_duration) AS session_duration
          FROM
            prodview_durations
          GROUP BY
            visitorId,product_name,
            itemId )
        SELECT
          *
        FROM
          aggregate_web_stats
    );
    -- Show table
    SELECT
      *
    FROM
      merch_store.aggregate_web_stats
"""
        

In [120]:
dd1 = bq2df(qq)

In [140]:
dd1.head()

,visitorId,product_name,itemId,session_duration
0,7963396423050396529-1,Maze Pen,GGOEGGOA017399,2881
1,6822031209977352236-1,Maze Pen,GGOEGGOA017399,21944
2,5558596643042634459-2,Maze Pen,GGOEGGOA017399,11565
3,1216652918682423609-1,Maze Pen,GGOEGGOA017399,2218
4,6694504690354356454-1,Maze Pen,GGOEGGOA017399,5711


In [138]:
dd1.agg(['count', 'size', 'nunique'])

,visitorId,product_name,itemId,session_duration
count,137515,137515,137515,137515
size,137515,137515,137515,137515
nunique,58531,443,374,38103


In [ ]:
dd1

In [87]:
qt = """
          SELECT
            CONCAT(fullVisitorID,'-', 
                 CAST(visitNumber AS STRING),'-', 
                 CAST(hitNumber AS STRING) ) AS visitorId_session_hit, ,time,
            LEAD(time, 1) OVER (
              PARTITION BY CONCAT(fullVisitorID,'-',CAST(visitNumber AS STRING))
              ORDER BY
              time ASC ) - time AS pageview_duration
          FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_2016*`,
            UNNEST(hits) AS hit 
"""

In [88]:
dt = bq2df(qt)

In [89]:
dt.head()

,visitorId_session_hit,f0_,time,pageview_duration
0,0003465185842457074-1-1,0003465185842457074-1,0,125744.0
1,0003465185842457074-1-2,0003465185842457074-1,125744,18909.0
2,0003465185842457074-1-3,0003465185842457074-1,144653,10122.0
3,0006293153380349462-1-1,0006293153380349462-1,0,87729.0
4,000722514342430295-1-1,000722514342430295-1,0,16622.0


In [126]:
qt1 = """    
    CREATE OR REPLACE TABLE merch_store.aggregate_web_stats AS (
      WITH
        durations AS (
          --calculate pageview durations
          SELECT
            CONCAT(fullVisitorID,'-', 
                 CAST(visitNumber AS STRING),'-', 
                 CAST(hitNumber AS STRING) ) AS visitorId_session_hit,
            LEAD(time, 1) OVER (
              PARTITION BY CONCAT(fullVisitorID,'-',CAST(visitNumber AS STRING))
              ORDER BY
              time ASC ) - time AS pageview_duration
          FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_2016*`,
            UNNEST(hits) AS hit 
        ),

        prodview_durations AS (
          --filter for product detail pages only
          SELECT
            CONCAT(fullVisitorID,'-',CAST(visitNumber AS STRING)) AS visitorId,
            productSKU AS itemId,
            IFNULL(dur.pageview_duration,
              1) AS pageview_duration,
          FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_2016*` t,
            UNNEST(hits) AS hits,
            UNNEST(hits.product) AS hits_product
          JOIN
            durations dur
          ON
            CONCAT(fullVisitorID,'-',
                   CAST(visitNumber AS STRING),'-',
                   CAST(hitNumber AS STRING)) = dur.visitorId_session_hit
          WHERE
          #action_type: Product detail views = 2
          eCommerceAction.action_type = "2" 
        ),

        aggregate_web_stats AS(
          --sum pageview durations by visitorId, itemId
          SELECT
            visitorId,
            itemId,
            SUM(pageview_duration) AS session_duration
          FROM
            prodview_durations
          GROUP BY
            visitorId,
            itemId )
        SELECT
          *
        FROM
          aggregate_web_stats
    );
    -- Show table
    SELECT
      *
    FROM
      merch_store.aggregate_web_stats

"""
        

In [127]:
dt1 = bq2df(qt1)

In [133]:
dt1.itemId.shape, dd1.itemId.shape

((137514,), (137515,))

In [ ]:
%%bigquery
CREATE TEMPORARY FUNCTION CLIP_LESS(x FLOAT64, a FLOAT64) AS (
  IF (x < a, a, x)
);
CREATE TEMPORARY FUNCTION CLIP_GT(x FLOAT64, b FLOAT64) AS (
  IF (x > b, b, x)
);
CREATE TEMPORARY FUNCTION CLIP(x FLOAT64, a FLOAT64, b FLOAT64) AS (
  CLIP_GT(CLIP_LESS(x, a), b)
);


CREATE OR REPLACE TABLE merch_store.recommendations_data 
AS 
WITH normalized_session_duration AS (
    SELECT APPROX_QUANTILES(session_duration,100)[OFFSET(50)] AS median_duration
    FROM merch_store.aggregate_web_stats
)
SELECT
   * EXCEPT(session_duration, median_duration),
   CLIP(0.3 * session_duration / median_duration, 0, 1.0) AS normalized_session_duration
FROM
   merch_store.aggregate_web_stats, normalized_session_duration



In [ ]:
df.agg(['count', 'size', 'nunique'])

In [ ]:
df = pd.read_csv(('../data/raw/raw.csv'))

In [ ]:
df.head()

In [3]:
import platform

In [4]:
platform.python_version()

'3.6.5'

In [1]:
from google.cloud import bigquery